In [59]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]

#Scale for better intepretation

data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

data.head(20)

/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_94927/1221689314.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_94927/1221689314.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_94927/1221689314.py:10: SettingWi

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,C,CH,257,1.0
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,C,CH,183,1.0
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,C,CH,177,1.0
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,C,CH,3,1.0
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,37.5,89.010141,40.2,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,37.5,80.003519,40.2,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,37.5,78.972366,40.2,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,37.5,67.303773,40.2,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,47.2,127.889611,52.2,17,Sedan,F,DE,84,1.0


In [61]:
data['HP**2'] = df['HP']**2
data['Range**2'] = df['Range']**2
data['Chargetime**2'] = df['Chargetime']**2

# create_instrument_localdiff(data, hvilket instrument, faktor std_dev skal ganges med (1 = 150 ca.))
# instrument hedder = instrument + '_instrument_localdiff'
fun.create_instrument_localdiff(data, 'HP', 1)
fun.create_instrument_localdiff(data, 'Chargetime', 1)
fun.create_instrument_localdiff(data, 'Range', 1)
data.head(20)

/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_94927/1073798670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HP**2'] = df['HP']**2
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_94927/1073798670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Range**2'] = df['Range']**2
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_94927/1073798670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const,HP**2,Range**2,Chargetime**2,HP_instrument_localdiff,Chargetime_instrument_localdiff,Range_instrument_localdiff
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,C,CH,257,1.0,40401,160000,1156,3811.9,3303,4242.9
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,C,CH,183,1.0,40401,160000,1156,3811.9,3303,4242.9
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,C,CH,177,1.0,40401,160000,1156,3811.9,3303,4242.9
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,C,CH,3,1.0,45796,164025,1156,3825.1,3303,4145.7
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,F,DE,222,1.0,161604,140625,289,4944.9,9367,5136.8
29,3,2020,0.034680,Audi,e-tron,37.5,89.010141,40.2,17,SUV,F,DE,491,1.0,161604,140625,289,4944.9,9367,5136.8
30,3,2021,0.010494,Audi,e-tron,37.5,80.003519,40.2,17,SUV,F,DE,260,1.0,161604,140625,289,4944.9,9367,5136.8
31,3,2022,0.017570,Audi,e-tron,37.5,78.972366,40.2,17,SUV,F,DE,538,1.0,161604,140625,289,4944.9,9367,5136.8
32,3,2023,0.001099,Audi,e-tron,37.5,67.303773,40.2,17,SUV,F,DE,68,1.0,161604,140625,289,4944.9,9367,5136.8
41,4,2021,0.003391,Audi,e-tron GT,47.2,127.889611,52.2,17,Sedan,F,DE,84,1.0,272484,222784,289,4630.0,9367,5270.6


In [62]:
# Copy the dataframe
df2 = data.copy()

In [63]:
def BLP(dataframe):
    # Group the DataFrame by the year
    grouped_data = dataframe.groupby('Year')

    # Define a function to sum the attributes of other models in the same year
    def sum_attributes(row):
        # Filter the DataFrame for the same year excluding the current model
        same_year_data = dataframe[(dataframe['Year'] == row['Year']) & (dataframe['Model'] != row['Model'])]
        
        # Sum the attributes of other models in the same year
        
        #Sum
        BLP_range = same_year_data['Range'].sum()
        BLP_hp = same_year_data['HP'].sum()
        BLP_chargetime = same_year_data['Chargetime'].sum()

        #Quadratic sum
        BLP_range2 = same_year_data['Range**2'].sum()
        BLP_hp2 = same_year_data['HP**2'].sum()
        BLP_chargetime2 = same_year_data['Chargetime**2'].sum()

        
        return pd.Series({'BLP_Range': BLP_range, 'BLP_HP': BLP_hp, 'BLP_Chargetime': BLP_chargetime})

    # Apply the function to each row in the DataFrame
    new_columns = dataframe.apply(sum_attributes, axis=1)

    # Add the new columns to the DataFrame
    dataframe['BLP_Range'] = new_columns['BLP_Range']
    dataframe['BLP_HP'] = new_columns['BLP_HP']
    dataframe['BLP_Chargetime'] = new_columns['BLP_Chargetime']

    return dataframe

# Call the function with your input DataFrame
df2 = BLP(df2)

# Print the updated DataFrame with summed attributes
print(df2)


       ID  Year  Market share Manufacturer   Model  Range      Price    HP  \
8       1  2021      0.010373       Aiways      U5   40.0  28.462170  20.1   
9       1  2022      0.005976       Aiways      U5   40.0  31.368183  20.1   
10      1  2023      0.002860       Aiways      U5   40.0  26.452400  20.1   
21      2  2023      0.000048       Aiways      U6   40.5  36.063800  21.4   
28      3  2019      0.040630         Audi  e-tron   37.5  97.970447  40.2   
...   ...   ...           ...          ...     ...    ...        ...   ...   
1154  189  2023      0.000065        Volvo    EX30   47.5  36.824500  26.8   
1163  190  2021      0.014652        Volvo    XC40   45.7  46.206060  40.2   
1164  190  2022      0.033310        Volvo    XC40   45.7  41.626340  40.2   
1165  190  2023      0.031752        Volvo    XC40   45.7  43.926660  40.2   
1176  192  2023      0.000129        Voyah    Free   50.0  50.476850  48.2   

      Chargetime Type  ... const   HP**2  Range**2  Chargetime*

In [64]:
# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)

In [65]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

# BLP Intrument (sum)

In [66]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'HP', 'Chargetime']]
dummies = df2[[ 'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['BLP_Range', 'BLP_HP', 'BLP_Chargetime']] 

In [67]:
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')

In [68]:
model.first_stage

,Price
R-squared,0.5822
Partial R-squared,0.0261
Shea's R-squared,0.0261
Partial F-statistic,17.145
P-value (Partial F-stat),0.0007
Partial F-stat Distn,chi2(3)
==========================,===========
const,-1.3281
,(-0.1735)
Range,0.1875


In [69]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -5.7877
Estimator:                    IV-2SLS   Adj. R-squared:                -5.8912
No. Observations:                 334   F-statistic:                    28.868
Date:                Sun, Apr 28 2024   P-value (F-stat)                0.0000
Time:                        12:42:13   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -2.8041     2.3134    -1.2121     0.2255     -7.3383      1.7300
Range          0.1995     0.0588     3.3923     0.0007      0.0842      0.3148
HP             0.3270     0.0955     3.4228     0.0006      0.1398      0.5143
Chargetime    -0.0848     0.0305    -2.7760     0.0055     -0.1447     -0.0249
China         -5.4735     1.1202    -4.8862     0.0000     -7.6690     -3.2779
Price         -0.3222     0.0761    -4.2313     0.0000     -0.4714     -0.1729
==============================================================================

Endogenous: Price
Instruments: BLP_Range, BLP_HP, BLP_Chargetime
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Gandhi Houde istrument (Quadratic diff)

In [70]:
z=df2[['Range**2', 'HP**2', 'Chargetime**2']] 
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')
model.first_stage

,Price
R-squared,0.6053
Partial R-squared,0.0799
Shea's R-squared,0.0799
Partial F-statistic,14.669
P-value (Partial F-stat),0.0021
Partial F-stat Distn,chi2(3)
==========================,===========
const,50.623
,(3.3376)
Range,-2.1731


In [71]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                      0.0084
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0067
No. Observations:                 334   F-statistic:                    74.802
Date:                Sun, Apr 28 2024   P-value (F-stat)                0.0000
Time:                        12:42:13   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -5.7644     0.8612    -6.6936     0.0000     -7.4523     -4.0765
Range          0.1047     0.0166     6.3014     0.0000      0.0721      0.1372
HP             0.0609     0.0333     1.8276     0.0676     -0.0044      0.1263
Chargetime    -0.0262     0.0135    -1.9457     0.0517     -0.0527      0.0002
China         -2.8189     0.4921    -5.7280     0.0000     -3.7834     -1.8544
Price         -0.0850     0.0259    -3.2828     0.0010     -0.1358     -0.0343
==============================================================================

Endogenous: Price
Instruments: Range**2, HP**2, Chargetime**2
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Gandhi Houde (Local Diff)

In [72]:
z=df2[['Range_instrument_localdiff', 'HP_instrument_localdiff', 'Chargetime_instrument_localdiff']] 
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')
model.first_stage

,Price
R-squared,0.5911
Partial R-squared,0.0468
Shea's R-squared,0.0468
Partial F-statistic,9.8961
P-value (Partial F-stat),0.0195
Partial F-stat Distn,chi2(3)
=================================,===========
const,-5.6265
,(-0.2761)
Range,0.5783


In [73]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                      0.0568
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0425
No. Observations:                 334   F-statistic:                    64.457
Date:                Sun, Apr 28 2024   P-value (F-stat)                0.0000
Time:                        12:42:13   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -5.8503     0.8972    -6.5205     0.0000     -7.6087     -4.0918
Range          0.1019     0.0176     5.7999     0.0000      0.0675      0.1363
HP             0.0532     0.0388     1.3708     0.1704     -0.0229      0.1293
Chargetime    -0.0245     0.0147    -1.6667     0.0956     -0.0534      0.0043
China         -2.7420     0.5328    -5.1461     0.0000     -3.7863     -1.6976
Price         -0.0782     0.0318    -2.4580     0.0140     -0.1405     -0.0158
==============================================================================

Endogenous: Price
Instruments: Range_instrument_localdiff, HP_instrument_localdiff, Chargetime_instrument_localdiff
Robust Covariance (Heteroskedastic)
Debiased: False
"""